In [8]:
from neural_network import datasets
from neural_network.classes.LossFunctions import MSE
from model_builder_monk import model_builder
from neural_network.classes.Callbacks import EarlyStopping
from neural_network.classes.Optimizers import *
from neural_network.classes.Validation import *
import pandas as pd
from neural_network.classes.ActivationFunctions import TanH
from neural_network.classes.Validation import TunerCV

In [9]:
hp = {"num_hidden_layers": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[2],
    unfold=True),
    "neurons_in_layer_1": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[5, 10, 20],
    unfold=True),
    "neurons_in_layer_2": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[5, 10, 20],
    unfold=True),
    "loss_function": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[MSE],
    unfold=True),
    "optimizer": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[SGD],
    unfold=True),
    "learning_rate": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[0.5, 0.3, 0.1, 0.05],
    unfold=True),
    "momentum": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[0.7, 0.4, 0.3],
    unfold=True),
    "regularization": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[0., 1e-7, 1e-6, 1e-5],
    unfold=True),
    "activation_function": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[TanH],
    unfold=True),
    "batch_size": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[50],
    unfold=True),
    "num_epochs": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[500],
    unfold=True),
    "callbacks": Hyperparameter(
    generator_logic="all_from_list",
    generator_space=[[EarlyStopping(monitor="val_mse", patience=50, mode="min", min_delta=1e-4, restore_best_weight=False)]],
    unfold=True)
}

In [10]:
dataset_attribute_columns = ["a1", "a2", "a3", "a4", "a5", "a6"]
dataset_class_column = "class"

paths = ["monk1_", "monk2_", "monk3_"]
for i, p in enumerate(paths):

    df, dft = datasets.read_monk(i+1)

    df = pd.get_dummies(df, columns=dataset_attribute_columns)
    dft = pd.get_dummies(dft, columns=dataset_attribute_columns)

    y = df.pop("class").to_numpy()
    X = df.to_numpy(dtype=np.float32)

    perm = np.random.permutation(X.shape[0])
    X = X[perm]
    y = y[perm]

    test_y = dft.pop("class").to_numpy()
    test_x = dft.to_numpy(dtype=np.float32)

    tuner = TunerCV(ConfigurationGenerator(hp, mode="grid"), model_builder, n_fold=4, verbose=True,
                    default_metric="val_binary_accuracy", default_reverse=True)

    val_res = tuner.fit(X, y)
    best_model = tuner.best_model()
    refit_results = best_model.fit(X, y)
    best_model.dump_weights("./" + p + "best_weights.pickle")
    test_result = best_model.evaluate_result(test_x, test_y)
    test_result.validation_results = val_res
    test_result.refit_results = refit_results
    test_result.dump("./" + p + "test_results.pickle")

Building model with the following configuration:Building model with the following configuration:Building model with the following configuration:Building model with the following configuration:   {'num_hidden_layers': 2, 'neurons_in_layer_1': 5, 'neurons_in_layer_2': 5, 'loss_function': <class 'neural_network.classes.LossFunctions.MSE.MSE'>, 'optimizer': <class 'neural_network.classes.Optimizers.SGD.SGD'>, 'learning_rate': 0.5, 'momentum': 0.7, 'regularization': 1e-07, 'activation_function': <class 'neural_network.classes.ActivationFunctions.TanH.TanH'>, 'batch_size': 50, 'num_epochs': 500, 'callbacks': [<neural_network.classes.Callbacks.EarlyStopping.EarlyStopping object at 0x7f56dd946b90>]}{'num_hidden_layers': 2, 'neurons_in_layer_1': 5, 'neurons_in_layer_2': 5, 'loss_function': <class 'neural_network.classes.LossFunctions.MSE.MSE'>, 'optimizer': <class 'neural_network.classes.Optimizers.SGD.SGD'>, 'learning_rate': 0.5, 'momentum': 0.7, 'regularization': 1e-06, 'activation_function':